In [24]:
from config import commodities
from CommidityData import Commodity_Data
from Data import Data
import pandas as pd
import statsmodels.api as sm
import numpy as np

In [25]:
dataset = {}
for gener, tickers in commodities.items():
    dataset[gener] = {}
    for key, ticker in tickers.items():
        temp = Data(ticker=ticker)
        temp.get_indicator_MOM(window=22)
        temp.get_indicator_TSMOM(window=22)
        temp.get_indicator_MOM(window=44)
        temp.get_indicator_TSMOM(window=44)
        temp.get_indicator_MOM(window=66)
        temp.get_indicator_TSMOM(window=66)
        temp.get_indicator_term()
        dataset[gener][key] = temp.data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [26]:
def get_factor_HML(x):
    mid = np.median(x)
    high_group = x[x > mid]
    low_group = x[x < mid]
    return high_group.mean() - low_group.mean()
    
factor_dict = {}
total_return_frame = pd.DataFrame()
for genre, ticker_frame in dataset.items():
    genre_return_frame = pd.DataFrame()
    for key, ticker in ticker_frame.items():
        total_return_frame[key] = ticker['return']
        genre_return_frame[key] = ticker['return']
    factor_dict[genre] = genre_return_frame.dropna().mean(axis=1)
factor_dict['market'] = total_return_frame.dropna().mean(axis=1)
total_return_frame['HML'] = total_return_frame.dropna().apply(lambda x: get_factor_HML(x), axis=1)
factor_dict['HML'] = total_return_frame['HML']

In [27]:
for genre, ticker_frame in dataset.items():
    for key, ticker in ticker_frame.items():
        ticker['genre'] = factor_dict[genre]
        ticker['market'] = factor_dict['market']
        ticker['HML'] = factor_dict['HML']

In [31]:
r_square = {}
for genre, ticker_frame in dataset.items():
    for key, ticker in ticker_frame.items():
        r_square[key] = {}
        for item in ['MOM', 'TSMOM', 'genre', 'market', "TERM", "HML"]:
            ticker = ticker.replace([np.inf, -np.inf], np.nan)
            ticker = ticker.dropna(subset=['return', 'TERM', 'MOM', 'HML',
                                       'genre', 'market', 'TSMOM'], how='any')
            #print(ticker)
            X = ticker[item]
            y = ticker['return']
            X = sm.add_constant(X)
            model = sm.OLS(y, X).fit()
            r_square[key][item] = model.rsquared_adj

In [28]:
from statsmodels.formula.api import ols
def forward_select(data, target):
    variate = set(data.columns)
    variate.remove(target)
    selected = []
    current_score, best_score = float('inf'), float('inf')
    while variate:
        aic_with_variate = []
        for candidate in variate:
            formula="{}~{}".format(target,"+".join(selected+[candidate]))
            aic=ols(formula=formula,data=data).fit().aic
            aic_with_variate.append((aic,candidate))
        aic_with_variate.sort(reverse=True)
        best_new_score, best_candidate=aic_with_variate.pop()
        if current_score>best_new_score:
            variate.remove(best_candidate)
            selected.append(best_candidate)
            current_score=best_new_score
            print("aic is {},continuing!".format(current_score))
        else:
            print("for selection over!")
            break
    formula="{}~{}".format(target,"+".join(selected))
    print("final formula is {}".format(formula))
    model=ols(formula=formula,data=data).fit()
    return(model)

In [29]:
result = {}
for genre, tickers in dataset.items():
    for key, ticker in tickers.items():
        print(key)
        ticker = ticker.dropna(subset=['return', 'TERM', 'MOM', 'HML', 'genre', 'market', 'TSMOM'], how='any')
        ticker = ticker[['return', 'TERM', 'MOM', 'HML', 'genre', 'market', 'TSMOM']]
        ticker.columns = ['ret', 'TERM', 'MOM', 'HML', 'genre', 'market', 'TSMOM']
        result[key] = forward_select(ticker, 'ret')

corn
aic is -20001.842096848643,continuing!
aic is -20075.602983042223,continuing!
aic is -20103.58205586999,continuing!
aic is -20104.537489298942,continuing!
for selection over!
final formula is ret~market+genre+TERM+MOM
rough rice
aic is -12224.764968333508,continuing!
aic is -12574.049988752184,continuing!
aic is -12659.841587116603,continuing!
aic is -12660.046793710859,continuing!
for selection over!
final formula is ret~genre+market+TERM+HML
wheat
aic is -19085.32560156298,continuing!
aic is -19154.33860303286,continuing!
aic is -19182.38398406065,continuing!
aic is -19190.46456694628,continuing!
aic is -19192.085672275753,continuing!
for selection over!
final formula is ret~market+genre+TERM+MOM+HML
soybean meal
aic is -19456.968685481635,continuing!
aic is -19527.422047785145,continuing!
aic is -19568.500900447576,continuing!
aic is -19571.55113828108,continuing!
aic is -19574.318503247705,continuing!
for selection over!
final formula is ret~market+genre+TERM+HML+MOM
soybean o

In [30]:
result

{'corn': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1420d70b8>,
 'rough rice': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1420cf828>,
 'wheat': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x141800940>,
 'soybean meal': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1420ec9b0>,
 'soybean oil': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x141566898>,
 'coffee': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x14159c9b0>,
 'cotton': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1418a8cf8>,
 'lumber': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x141211080>,
 'sugar': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x13e663630>,
 'silver': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1421054a8>,
 'gold': <statsmodels.regression.linear_model.RegressionResultsWrapper at 0x1418aeb38>,
 

In [31]:
for key, value in result.items():
    print(value.summary())

                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.197
Model:                            OLS   Adj. R-squared:                  0.196
Method:                 Least Squares   F-statistic:                     224.0
Date:                Fri, 27 Nov 2020   Prob (F-statistic):          3.32e-172
Time:                        03:16:01   Log-Likelihood:                 10057.
No. Observations:                3666   AIC:                        -2.010e+04
Df Residuals:                    3661   BIC:                        -2.007e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0006      0.000      1.350      0.1

                            OLS Regression Results                            
Dep. Variable:                    ret   R-squared:                       0.350
Model:                            OLS   Adj. R-squared:                  0.349
Method:                 Least Squares   F-statistic:                     394.3
Date:                Fri, 27 Nov 2020   Prob (F-statistic):               0.00
Time:                        03:16:01   Log-Likelihood:                 8310.5
No. Observations:                3666   AIC:                        -1.661e+04
Df Residuals:                    3660   BIC:                        -1.657e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0024      0.002      1.538      0.1